# 图像细粒度分类

Inception、Xception、NASNetLarge、InceptionRes in Tensorflow2

Kaggle Competition:Dog Breed Identification

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout, Lambda, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img

# 数据集处理

数据主要结构是训练集图像放在训练集文件夹里面，标签以csv格式的数据记录

In [ ]:
# 读取标签
labels = pd.read_csv('./dog-breed/labels.csv')
print(labels.head())
print(labels.describe())

In [ ]:
# 查看数据的格式
#Create list of alphabetically sorted labels.
classes = sorted(list(set(labels['breed'])))
n_classes = len(classes)
print('Total unique breed {}'.format(n_classes))

#Map each label string to an integer label.
class_to_num = dict(zip(classes, range(n_classes)))
class_to_num

In [ ]:
# 设置成331的原因是NASNetLarge默认的输入shape大小是(331, 331, 3)
img_size = (331, 331, 3)

def images_to_array(directory, label_dataframe, target_size = img_size):
    image_labels = label_dataframe['breed']
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]],dtype=np.uint8)
    y = np.zeros([len(label_dataframe),1],dtype = np.uint8)
    for ix, image_name in enumerate(label_dataframe['id'].values):
        img_dir = os.path.join(directory, image_name+'.jpg')
        img = load_img(img_dir, target_size=target_size)
        images[ix]=img
        del img

        dog_breed = image_labels[ix]
        y[ix] = class_to_num[dog_breed]
    y = to_categorical(y)
    return images, y

In [ ]:
import time 
t = time.time()
X,y = images_to_array('./dog-breed/train', labels[:])
print('runtime in seconds: {}'.format(time.time() - t))

In [ ]:
# 展示数据
import matplotlib.pyplot as plt
n=25
plt.figure(figsize=(20,20))

for i in range(n):
    ax = plt.subplot(5, 5, i+1)
    plt.title(classes[np.where(y[i] ==1)[0][0]])
    plt.imshow(X[i].astype('int32')) 

## 建立模型
建立Inception、Xception、NASNetLarge, InceptionRes

In [2]:

def get_feature(model_name, model_preprocessor, input_size, data):
    input_layer = Input(img_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False, input_shape = input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs=input_layer, outputs=avg)
    # Extract feature
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape:', feature_maps.shape)
    return feature_maps

In [ ]:
# Extract featurs using InceptionV3
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_feature(InceptionV3, inception_preprocessor, img_size, X)

In [ ]:
# Extract features using Xception 
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_feature(Xception,
                                 xception_preprocessor,
                                 img_size, X)

In [ ]:
# Extract features using InceptionResNetV2 
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_feature(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X)

In [ ]:
# Extract features using NASNetLarge 
from keras.applications.nasnet import NASNetLarge, preprocess_input
nasnet_preprocessor = preprocess_input
nasnet_features = get_feature(NASNetLarge,
                               nasnet_preprocessor,
                               img_size, X)

In [ ]:
# Hyperparameters
batch_size= 128
epochs=50
learn_rate=.001
sgd=tf.keras.optimizers.SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam= tf.keras.optimizers.Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None,  amsgrad=False)

In [ ]:
#Learning Rate Annealer
lrr= ReduceLROnPlateau(monitor='val_acc', factor=.01, patience=3, min_lr=1e-5,verbose = 1)

#Prepare call backs
EarlyStop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
#Creating final featuremap by combining all extracted features

final_features = np.concatenate([inception_features,
                                 xception_features,
                                #  nasnet_features,
                                 inc_resnet_features], axis=-1) #axis=-1 to concatinate horizontally

print('Final feature maps shape', final_features.shape)

In [ ]:
#Prepare Deep net

model = Sequential()
# model.add(Dense(1028,input_shape=(final_features.shape[1],)))
model.add(Dropout(0.7,input_shape=(final_features.shape[1],)))
model.add(Dense(n_classes,activation= 'softmax'))

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Training the model. 
history = model.fit(final_features, y,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
            callbacks=[lrr,EarlyStop])

# 测试模型

In [ ]:
img_g = load_img('./dog-breed/train/0a0c223352985ec154fd604d7ddceabd.jpg',target_size = img_size)
img_g = np.expand_dims(img_g, axis=0)

In [ ]:
# #Predict test labels given test data features.
test_features = extact_features(img_g)
predg = model.predict(test_features)
print(f"Predicted label: {classes[np.argmax(predg[0])]}")
print(f"Probability of prediction): {round(np.max(predg[0])) * 100} %")